In [4]:
## previous notebook had gotten out of hand - reimporting the processing with only key aspects so I can focus on modeling

In [46]:
from datetime import datetime, timedelta
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy.stats as stats
import seaborn as sns
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline

import networkx as nx
from networkx.algorithms.components.connected import connected_components

%matplotlib notebook
%timeit

#show up to 100 columns.
pd.set_option('display.max_columns', 100)

In [6]:
data_file = "new_theorem_data.p"

## data appears to have been savied in Python 2 - changing encoding allows us to properly load data
with open(data_file, 'rb') as pickle_file:
    data = pickle.load(pickle_file, encoding='latin1') 

In [40]:
## find correlation of each of the variables with one another
corr = data.corr()

## 86 different categories - start by taking correlation matrix to figure out which categories are redundant
f, ax = plt.subplots(figsize=(12, 9))

# Draw the heatmap using seaborn
sns.corrplot(corr, square=True)

AttributeError: module 'seaborn' has no attribute 'corrplot'

In [8]:
## thank you to stack overflow for the elegant solution: 
## http://stackoverflow.com/questions/4842613/merge-lists-that-share-common-elements
def to_graph(l):
    G = nx.Graph()
    for part in l:
        G.add_nodes_from(part)
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    """ 
        treat `l` as a Graph and returns it's edges 
        to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
    """
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current    

In [9]:
## cutoff of .6 recovers 47 inter-connected variables, .5 recovers 52, .55 recovers 50
## cutoff of .55 features largest group at length 10 - probably don't want to exceed that
tentative_families = []
cutoff = .55

for name, col in corr.iteritems():
    
    highly_correlated = (abs(col) > cutoff) & (col.index != name)
    high_corr_list = col[highly_correlated].index.tolist()
    
    if high_corr_list != []:
        tentative_families.append(sorted(high_corr_list + [ name ]))
        
tentative_families.sort()
tentative_families = list(tentative_families for tentative_families,_ in itertools.groupby(tentative_families))

for fam in sorted(tentative_families):
    print(fam)

['BoolEverWholeLoan', 'BoolIsFractionalLoan', 'EnumLoanFractionalType']
['BoolOwnsHome', 'DolMonthlyDebt', 'DolRealEstateBalance', 'DolRealEstatePayment', 'DolTotalBalanceAllOpenTrades6', 'DolTotalPaymentAllOpenTrades6']
['BoolOwnsHome', 'DolRealEstateBalance', 'DolRealEstatePayment', 'DolTotalBalanceAllOpenTrades6', 'DolTotalPaymentAllOpenTrades6']
['BoolOwnsHome', 'DolRealEstateBalance', 'DolTotalBalanceAllOpenTrades6', 'DolTotalPaymentAllOpenTrades6']
['BorrowerRate', 'LenderYield', 'ProsperScore']
['DolLoanAmountRequested', 'DolMonthlyLoanPayment']
['DolMaxPriorProsperLoan', 'DolMinPriorProsperLoan', 'DolPriorProsperLoansBalanceOutstanding', 'DolPriorProsperLoansPrincipalBorrowed', 'DolPriorProsperLoansPrincipalOutstanding']
['DolMaxPriorProsperLoan', 'DolMinPriorProsperLoan', 'DolPriorProsperLoansBalanceOutstanding', 'DolPriorProsperLoansPrincipalOutstanding']
['DolMaxPriorProsperLoan', 'DolMinPriorProsperLoan', 'DolPriorProsperLoansBalanceOutstanding', 'DolPriorProsperLoansPrinci

In [10]:
## key code that combines lists that share a common element.
G = to_graph(tentative_families)
cc = nx.connected_components(G)
families = []

for nodes in cc:
    families.append(list(nodes))
    
families = sorted([ sorted(fam) for fam in families ])
    
for fam in families:
    print(fam, len(fam))
    
ll = [ len(fam) for fam in families ]
print(sum(ll))

['BoolEverWholeLoan', 'BoolIsFractionalLoan', 'EnumLoanFractionalType'] 3
['BoolOwnsHome', 'DolMonthlyDebt', 'DolRealEstateBalance', 'DolRealEstatePayment', 'DolTotalBalanceAllOpenTrades6', 'DolTotalBalanceInstallTradesReptd6', 'DolTotalPaymentAllOpenTrades6'] 7
['BorrowerRate', 'LenderYield', 'ProsperScore'] 3
['DolLoanAmountRequested', 'DolMonthlyLoanPayment'] 2
['DolMaxPriorProsperLoan', 'DolMinPriorProsperLoan', 'DolPriorProsperLoansBalanceOutstanding', 'DolPriorProsperLoansPrincipalBorrowed', 'DolPriorProsperLoansPrincipalOutstanding', 'NumPriorProsperLoans', 'NumPriorProsperLoansActive', 'NumPriorProsperLoansCyclesBilled', 'NumPriorProsperLoansEarliestPayOff', 'NumPriorProsperLoansOnTimePayments'] 10
['DolRevolvingBalance', 'DolTotalBalanceOpenRevolving6'] 2
['NumBankcardTradesOpened12', 'NumCreditLines84', 'NumCurrentCreditLines', 'NumOpenCreditLines', 'NumOpenRevolvingAccounts', 'NumSatisfactoryAccounts', 'NumTrades'] 7
['NumDelinquencies84', 'NumDelinquenciesOver30Days', 'NumD

In [11]:
## give custom descriptive names to each family
# two items: first, each unit in the family; second, the variable(s) you're going to keep around.
family_dict = {}
family_dict['fractional_loan'] = [families[0],'BoolEverWholeLoan']
family_dict['debt'] = [families[1],'DolMonthlyDebt']
family_dict['borrower_rate'] = [families[2],'BorrowerRate']
family_dict['loan_amount'] = [families[3],'DolLoanAmountRequested']
family_dict['prosper_history'] = [families[4],'DolPriorProsperLoansPrincipalBorrowed']
family_dict['revolving_balance'] = [families[5],'DolTotalBalanceOpenRevolving6']
family_dict['credit'] = [families[6],'NumCurrentCreditLines']
family_dict['delinquencies'] = [families[7],'PctTradesNeverDelinquent']
family_dict['inquiries'] = [families[8],'NumTotalInquiries']
family_dict['prior_prosper_loans'] = [families[9],'NumPriorProsperLoans61dpd']
family_dict['real_estate'] = [families[10],'NumRealEstateTrades']
family_dict['current_delinquency'] = [families[11],'NumTradesCurr30DPDOrDerog6']

In [12]:
## create reduced data set by dropping headers...
reduced_data = pd.DataFrame.copy(data, deep = True)

## immediately drop columns that are unlikely/unable to cause loan cancellation (ID numbers/dates)
drop_cols = ['ListingID', 'DateCreditPulled', 'DateListingStart', 'DateListingCreation', 'DateWholeLoanStart', 'DateWholeLoanEnd']
reduced_data.drop(drop_cols, axis = 1, inplace = True)

In [13]:
## drop similar columns as determined by families
for fam, items in family_dict.items():
    drop_cols = list(items[0])
    drop_cols.remove(items[1]) #keep our chosen representative variable for that family
    reduced_data.drop(drop_cols, axis = 1, inplace = True)

In [14]:
## manually drop a few other columns that won't help our analysis...
reduced_data.drop(['StrState','StrBorrowerCity','BoolIncomeVerifiable'], axis=1, inplace=True)

In [15]:
## replace DateFirstCredit column with an integer
days_since_firstcredit = datetime.now()-reduced_data['DateFirstCredit']
reduced_data['DaysSinceFirstCredit'] = [ x.days for x in days_since_firstcredit ]
reduced_data.drop('DateFirstCredit', axis=1, inplace=True)

In [16]:
## quick plot to show that BorrowerRate and CreditGrade are pretty much equivalent - drop CreditGrade
reduced_data['BorrowerRate'].hist(by=reduced_data['CreditGrade'])
reduced_data.drop('CreditGrade',axis =1, inplace=True)

In [17]:
## Deal with remaining categorical variables - StrEmploymentStatus,StrOccupation,BoolInGroup and BoolEverWholeLoan
#pd.get_dummies(reduced_data['CreditGrade'])
label_encoder = preprocessing.LabelEncoder()

# Convert Sex variable to numeric
reduced_data["BoolInGroup"] = label_encoder.fit_transform(reduced_data["BoolInGroup"])
reduced_data["BoolEverWholeLoan"] = label_encoder.fit_transform(reduced_data["BoolEverWholeLoan"])
reduced_data.head()

,EnumListingStatus,DolLoanAmountRequested,BoolPartialFundingApproved,BorrowerRate,NumMonthsTerm,FICOScore,EnumListingCategory,DolMonthlyIncome,FracDebtToIncomeRatio,StrEmploymentStatus,StrOccupation,NumMonthsEmployed,DolPriorProsperLoansPrincipalBorrowed,NumPriorProsperLoansLateCycles,NumPriorProsperLoansLatePayments,NumPriorProsperLoans61dpd,BoolIsLender,BoolInGroup,EnumChannelCode,NumTradesOpened6,NumOpenTradesDelinqOrPastDue6,NumTradesCurr30DPDOrDerog6,DolTotalBalanceOnPublicRecords,AgeOldestTrade,PctTradesNeverDelinquent,DolTotalAvailBankcardCredit6,NumRealEstateTrades,DolTotalBalanceOpenRevolving6,DolMonthlyDebt,NumCurrentDelinquencies,NumPublicRecordsLast10Years,NumPublicRecords12,DolAmountDelinquent,NumCurrentCreditLines,PctBankcardUtil,NumTotalInquiries,BoolEverWholeLoan,DaysSinceFirstCredit
ListingNumber,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
973605,7,15000.0,True,0.1620,60,689,1,6000.0000,0.27,Employed,Tradesman - Mechanic,445.0,NaN,NaN,NaN,NaN,0,0,70000,1,0,0,0,419,96,439,2,33956,1242,0,0,0,0,11,0.97,5,1,13899
981099,7,15000.0,True,0.1585,60,709,1,7916.6667,0.35,Other,NaN,32.0,NaN,NaN,NaN,NaN,0,0,70000,1,0,0,0,430,100,9609,2,14852,2289,0,0,0,0,15,0.48,3,1,14236
1025766,6,4000.0,True,0.2085,36,669,1,2083.3333,0.53,Employed,Professional,4.0,14400.0,0.0,0.0,NaN,0,0,80000,0,0,0,0,100,86,1498,0,20802,911,0,0,0,0,8,0.93,5,0,4144
1003835,7,10000.0,True,0.1299,36,749,13,3750.0000,0.14,Employed,Medical Technician,2.0,NaN,NaN,NaN,NaN,0,0,90000,1,0,0,0,60,100,6050,0,3100,223,0,0,0,0,7,0.26,1,1,2940
1011335,6,20000.0,True,0.1440,60,649,1,9000.0000,0.16,Employed,Executive,90.0,2600.0,2.0,0.0,NaN,0,0,80000,1,0,0,1249,237,90,739,1,5519,1264,1,2,0,0,11,0.81,17,0,8327


In [18]:
## the probability of loan cancellation actually seems to vary a lot by profession
jobs = reduced_data.groupby('StrOccupation')
jobs['EnumListingStatus'].mean()[0:]

StrOccupation
Accountant/CPA                        6.289733
Administrative Assistant              6.305882
Analyst                               6.230218
Architect                             6.291925
Attorney                              6.319324
Biologist                             6.303191
Bus Driver                            6.424473
Car Dealer                            6.391241
Chemist                               6.314465
Civil Service                         6.231803
Clergy                                6.456621
Clerical                              6.309499
Computer Programmer                   6.227259
Construction                          6.397276
Dentist                               6.505556
Doctor                                6.417600
Engineer - Chemical                   6.364865
Engineer - Electrical                 6.269855
Engineer - Mechanical                 6.289251
Executive                             6.340349
Fireman                               6.290365

In [19]:
## some professions are more likely to be self-employed than others, but not a one-to-one match
jobs['StrEmploymentStatus'].value_counts(normalize = True)

StrOccupation                StrEmploymentStatus
Accountant/CPA               Employed               0.928640
                             Self-employed          0.065162
                             Full-time              0.003179
                             Other                  0.003020
Administrative Assistant     Employed               0.973428
                             Self-employed          0.018661
                             Full-time              0.006085
                             Other                  0.001826
Analyst                      Employed               0.983428
                             Self-employed          0.014631
                             Full-time              0.001792
                             Other                  0.000149
Architect                    Employed               0.902692
                             Self-employed          0.084886
                             Full-time              0.006211
                             Other  

In [20]:
## Handle NaN values with sklearn's imputer - DolPriorProsperLoansPrincipalBorrowed, NumPriorProsperLoansLateCycles,
#NumPriorProsperLoansLatePayments and NumPriorProsperLoans61dpd

print(reduced_data.isnull().any())

## two more columns feature a small amount of NaNs: FracDebtToIncomeRatio and NumMonthsEmployed

##from looking at NumPriorProsperLoans61dpd, probably need to handle that variable differently - only 50 non-Nan values
## handle the first 3 variables together first.
imp = preprocessing.Imputer(missing_values="NaN",strategy='mean',axis=0)

#replace nans with column mean
test_out = imp.fit_transform(reduced_data[['DolPriorProsperLoansPrincipalBorrowed', 'NumPriorProsperLoansLateCycles',\
                     'NumPriorProsperLoansLatePayments','FracDebtToIncomeRatio','NumMonthsEmployed']])

reduced_data['DolPriorProsperLoansPrincipalBorrowed'] = test_out[:,0]
reduced_data['NumPriorProsperLoansLateCycles'] = test_out[:,1]
reduced_data['NumPriorProsperLoansLatePayments'] = test_out[:,2]
reduced_data['FracDebtToIncomeRatio'] = test_out[:,3]
reduced_data['NumMonthsEmployed'] = test_out[:,4]

#Having a loan over 61 days past due is a rare event - should expect 0, not mean of actual observations!
reduced_data['NumPriorProsperLoans61dpd'].fillna(0, inplace = True)

EnumListingStatus                        False
DolLoanAmountRequested                   False
BoolPartialFundingApproved               False
BorrowerRate                             False
NumMonthsTerm                            False
FICOScore                                False
EnumListingCategory                      False
DolMonthlyIncome                         False
FracDebtToIncomeRatio                     True
StrEmploymentStatus                      False
StrOccupation                             True
NumMonthsEmployed                         True
DolPriorProsperLoansPrincipalBorrowed     True
NumPriorProsperLoansLateCycles            True
NumPriorProsperLoansLatePayments          True
NumPriorProsperLoans61dpd                 True
BoolIsLender                             False
BoolInGroup                              False
EnumChannelCode                          False
NumTradesOpened6                         False
NumOpenTradesDelinqOrPastDue6            False
NumTradesCurr

In [21]:
#check if there are still NaN values left
reduced_data.isnull().any()

EnumListingStatus                        False
DolLoanAmountRequested                   False
BoolPartialFundingApproved               False
BorrowerRate                             False
NumMonthsTerm                            False
FICOScore                                False
EnumListingCategory                      False
DolMonthlyIncome                         False
FracDebtToIncomeRatio                    False
StrEmploymentStatus                      False
StrOccupation                             True
NumMonthsEmployed                        False
DolPriorProsperLoansPrincipalBorrowed    False
NumPriorProsperLoansLateCycles           False
NumPriorProsperLoansLatePayments         False
NumPriorProsperLoans61dpd                False
BoolIsLender                             False
BoolInGroup                              False
EnumChannelCode                          False
NumTradesOpened6                         False
NumOpenTradesDelinqOrPastDue6            False
NumTradesCurr

In [22]:
reduced_data.head()

,EnumListingStatus,DolLoanAmountRequested,BoolPartialFundingApproved,BorrowerRate,NumMonthsTerm,FICOScore,EnumListingCategory,DolMonthlyIncome,FracDebtToIncomeRatio,StrEmploymentStatus,StrOccupation,NumMonthsEmployed,DolPriorProsperLoansPrincipalBorrowed,NumPriorProsperLoansLateCycles,NumPriorProsperLoansLatePayments,NumPriorProsperLoans61dpd,BoolIsLender,BoolInGroup,EnumChannelCode,NumTradesOpened6,NumOpenTradesDelinqOrPastDue6,NumTradesCurr30DPDOrDerog6,DolTotalBalanceOnPublicRecords,AgeOldestTrade,PctTradesNeverDelinquent,DolTotalAvailBankcardCredit6,NumRealEstateTrades,DolTotalBalanceOpenRevolving6,DolMonthlyDebt,NumCurrentDelinquencies,NumPublicRecordsLast10Years,NumPublicRecords12,DolAmountDelinquent,NumCurrentCreditLines,PctBankcardUtil,NumTotalInquiries,BoolEverWholeLoan,DaysSinceFirstCredit
ListingNumber,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
973605,7,15000.0,True,0.1620,60,689,1,6000.0000,0.27,Employed,Tradesman - Mechanic,445.0,12310.094987,0.359752,0.03088,0.0,0,0,70000,1,0,0,0,419,96,439,2,33956,1242,0,0,0,0,11,0.97,5,1,13899
981099,7,15000.0,True,0.1585,60,709,1,7916.6667,0.35,Other,NaN,32.0,12310.094987,0.359752,0.03088,0.0,0,0,70000,1,0,0,0,430,100,9609,2,14852,2289,0,0,0,0,15,0.48,3,1,14236
1025766,6,4000.0,True,0.2085,36,669,1,2083.3333,0.53,Employed,Professional,4.0,14400.000000,0.000000,0.00000,0.0,0,0,80000,0,0,0,0,100,86,1498,0,20802,911,0,0,0,0,8,0.93,5,0,4144
1003835,7,10000.0,True,0.1299,36,749,13,3750.0000,0.14,Employed,Medical Technician,2.0,12310.094987,0.359752,0.03088,0.0,0,0,90000,1,0,0,0,60,100,6050,0,3100,223,0,0,0,0,7,0.26,1,1,2940
1011335,6,20000.0,True,0.1440,60,649,1,9000.0000,0.16,Employed,Executive,90.0,2600.000000,2.000000,0.00000,0.0,0,0,80000,1,0,0,1249,237,90,739,1,5519,1264,1,2,0,0,11,0.81,17,0,8327


In [23]:
## let's parse out highly correlated variables one more time to get to reasonable set
reduced_corr = reduced_data.corr()

f, ax = plt.subplots(figsize=(12, 9))

# Draw the heatmap using seaborn
sns.heatmap(reduced_corr, square=True)

In [24]:
## cutoffs of .5, .45 and .4 all retrieve pretty similar stuff on second pass. let's stick with .45.
tentative_families = []
cutoff = .45

for name, col in reduced_corr.iteritems():
    
    highly_correlated = (abs(col) > cutoff) & (col.index != name)
    high_corr_list = col[highly_correlated].index.tolist()
    
    if high_corr_list != []:
        tentative_families.append(sorted(high_corr_list + [ name ]))
        
tentative_families.sort()
tentative_families = list(tentative_families for tentative_families,_ in itertools.groupby(tentative_families))

for fam in sorted(tentative_families):
    print(fam)

['AgeOldestTrade', 'DaysSinceFirstCredit']
['BorrowerRate', 'DolTotalAvailBankcardCredit6', 'FICOScore', 'PctTradesNeverDelinquent']
['BorrowerRate', 'FICOScore']
['DolMonthlyDebt', 'DolTotalBalanceOpenRevolving6']
['DolMonthlyDebt', 'DolTotalBalanceOpenRevolving6', 'NumCurrentCreditLines']
['DolMonthlyDebt', 'NumCurrentCreditLines']
['DolTotalAvailBankcardCredit6', 'FICOScore']
['FICOScore', 'PctTradesNeverDelinquent']
['NumCurrentDelinquencies', 'NumTradesCurr30DPDOrDerog6']
['NumPriorProsperLoans61dpd', 'NumPriorProsperLoansLateCycles', 'NumPriorProsperLoansLatePayments']
['NumPriorProsperLoans61dpd', 'NumPriorProsperLoansLatePayments']
['NumPriorProsperLoansLateCycles', 'NumPriorProsperLoansLatePayments']


In [33]:
G = to_graph(tentative_families)
cc = nx.connected_components(G)
new_families = []

for nodes in cc:
    new_families.append(list(nodes))
    
new_families = sorted([ sorted(fam) for fam in new_families ])
    
for fam in new_families:
    print(fam, len(fam))
    
ll = [ len(fam) for fam in new_families ]
print(sum(ll))

['AgeOldestTrade', 'DaysSinceFirstCredit'] 2
['BorrowerRate', 'DolTotalAvailBankcardCredit6', 'FICOScore', 'PctTradesNeverDelinquent'] 4
['DolMonthlyDebt', 'DolTotalBalanceOpenRevolving6', 'NumCurrentCreditLines'] 3
['NumCurrentDelinquencies', 'NumTradesCurr30DPDOrDerog6'] 2
['NumPriorProsperLoans61dpd', 'NumPriorProsperLoansLateCycles', 'NumPriorProsperLoansLatePayments'] 3
14


In [34]:
## give custom descriptive names to each family
# two items: first, each unit in the family; second, the variable(s) you're going to keep around.
family_dict_2 = {}
family_dict_2['oldest_trade'] = [new_families[0],'DaysSinceFirstCredit']
family_dict_2['borrower_rate'] = [new_families[1],'BorrowerRate']
family_dict_2['monthly_debt'] = [new_families[2],'DolMonthlyDebt']
family_dict_2['current_delinquencies'] = [new_families[3],'NumCurrentDelinquencies']
family_dict_2['prosper_delinquencies'] = [new_families[4],'NumPriorProsperLoansLatePayments']

In [35]:
## again, drop similar columns as determined by families
for fam, items in family_dict_2.items():
    drop_cols = list(items[0])
    drop_cols.remove(items[1]) #keep our chosen representative variable for that family
    reduced_data.drop(drop_cols, axis = 1, inplace = True)

In [36]:
print("Number of columns:",len(reduced_data.columns.values))
reduced_data.head()

Number of columns: 29


,EnumListingStatus,DolLoanAmountRequested,BoolPartialFundingApproved,BorrowerRate,NumMonthsTerm,EnumListingCategory,DolMonthlyIncome,FracDebtToIncomeRatio,StrEmploymentStatus,StrOccupation,NumMonthsEmployed,DolPriorProsperLoansPrincipalBorrowed,NumPriorProsperLoansLatePayments,BoolIsLender,BoolInGroup,EnumChannelCode,NumTradesOpened6,NumOpenTradesDelinqOrPastDue6,DolTotalBalanceOnPublicRecords,NumRealEstateTrades,DolMonthlyDebt,NumCurrentDelinquencies,NumPublicRecordsLast10Years,NumPublicRecords12,DolAmountDelinquent,PctBankcardUtil,NumTotalInquiries,BoolEverWholeLoan,DaysSinceFirstCredit
ListingNumber,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
973605,7,15000.0,True,0.1620,60,1,6000.0000,0.27,Employed,Tradesman - Mechanic,445.0,12310.094987,0.03088,0,0,70000,1,0,0,2,1242,0,0,0,0,0.97,5,1,13899
981099,7,15000.0,True,0.1585,60,1,7916.6667,0.35,Other,NaN,32.0,12310.094987,0.03088,0,0,70000,1,0,0,2,2289,0,0,0,0,0.48,3,1,14236
1025766,6,4000.0,True,0.2085,36,1,2083.3333,0.53,Employed,Professional,4.0,14400.000000,0.00000,0,0,80000,0,0,0,0,911,0,0,0,0,0.93,5,0,4144
1003835,7,10000.0,True,0.1299,36,13,3750.0000,0.14,Employed,Medical Technician,2.0,12310.094987,0.03088,0,0,90000,1,0,0,0,223,0,0,0,0,0.26,1,1,2940
1011335,6,20000.0,True,0.1440,60,1,9000.0000,0.16,Employed,Executive,90.0,2600.000000,0.00000,0,0,80000,1,0,1249,1,1264,1,2,0,0,0.81,17,0,8327


In [99]:
model = linear_model.LogisticRegression(C=.000001)
x = reduced_data['DolLoanAmountRequested'].to_frame()
y = reduced_data['EnumListingStatus'].to_frame()-6
mm = model.fit(x,y)
x_range = np.linspace(int(x.min()),int(y.min()),200).reshape([200,1])
probs = mm.predict_proba(x_range)
predicts = mm.predict(x_range)

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [100]:
y['EnumListingStatus'].value_counts()

0    168052
1     84417
Name: EnumListingStatus, dtype: int64

In [101]:
print(probs[:100])

[[ 0.51669851  0.48330149]
 [ 0.5166147   0.4833853 ]
 [ 0.51653089  0.48346911]
 [ 0.51644708  0.48355292]
 [ 0.51636327  0.48363673]
 [ 0.51627945  0.48372055]
 [ 0.51619564  0.48380436]
 [ 0.51611182  0.48388818]
 [ 0.51602801  0.48397199]
 [ 0.51594419  0.48405581]
 [ 0.51586037  0.48413963]
 [ 0.51577655  0.48422345]
 [ 0.51569274  0.48430726]
 [ 0.51560892  0.48439108]
 [ 0.51552509  0.48447491]
 [ 0.51544127  0.48455873]
 [ 0.51535745  0.48464255]
 [ 0.51527363  0.48472637]
 [ 0.5151898   0.4848102 ]
 [ 0.51510598  0.48489402]
 [ 0.51502215  0.48497785]
 [ 0.51493832  0.48506168]
 [ 0.51485449  0.48514551]
 [ 0.51477067  0.48522933]
 [ 0.51468684  0.48531316]
 [ 0.51460301  0.48539699]
 [ 0.51451917  0.48548083]
 [ 0.51443534  0.48556466]
 [ 0.51435151  0.48564849]
 [ 0.51426768  0.48573232]
 [ 0.51418384  0.48581616]
 [ 0.51410001  0.48589999]
 [ 0.51401617  0.48598383]
 [ 0.51393233  0.48606767]
 [ 0.5138485   0.4861515 ]
 [ 0.51376466  0.48623534]
 [ 0.51368082  0.48631918]
 

In [69]:
x = reduced_data['DolLoanAmountRequested'].to_frame()
int(x.min())

2000

In [59]:
mm.predict_proba(linspace())

[[ -3.33931665e-05]]


In [57]:
reduced_data['DolLoanAmountRequested'].shape

(252469,)

In [56]:
sns.set(style="darkgrid")
x = reduced_data['DolLoanAmountRequested']
y = reduced_data['EnumListingStatus']-6
clf = linear_model.LogisticRegression().fit(x, y)


fig, ax = plt.subplots()

plt.scatter(x[:10000],y[:10000])

# Show the survival proability as a function of age and sex
#g = sns.lmplot(x="DolLoanAmountRequested", y="EnumListingStatus", data=reduced_data, logistic=True)
#g.set(xlim=(0, 80), ylim=(-.05, 1.05))

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Found arrays with inconsistent numbers of samples: [     1 252469]